In [1]:
# Import some standard/utility libraries:
import os, sys, six # six provides python 2/3 compatibility

# Import our numerical/scientific libraries, scipy and numpy:
import numpy as np
import scipy as sp
import nibabel as nib
from os.path import join as pjoin
import h5py
from nod_utils import save_ciftifile
# The neuropythy library is a swiss-army-knife for handling MRI data, especially
# anatomical/structural data such as that produced by FreeSurfer or the HCP.
# https://github.com/noahbenson/neuropythy
import neuropythy as ny

# Import graphics libraries:
# Matplotlib/Pyplot is our 2D graphing library:
import matplotlib as mpl
import matplotlib.pyplot as plt

# We also import ipyvolume, the 3D graphics library used by neurropythy, for 3D
# surface rendering (optional).
import ipyvolume as ipv

import cv2
import nibabel as nib
import numpy as np

from nibabel.cifti2 import cifti2
from collections import OrderedDict



In [2]:
class CiftiReader(object):

    def __init__(self, file_path):
        self.full_data = cifti2.load(file_path)

    @property
    def header(self):
        return self.full_data.header

    @property
    def brain_structures(self):
        return [_.brain_structure for _ in self.header.get_index_map(1).brain_models]

    @property
    def label_info(self):
        """
        Get label information from label tables
        Return:
        ------
        label_info[list]:
            Each element is a dict about corresponding map's label information.
            Each dict's content is shown as below:
                key[list]: a list of integers which are data values of the map
                label[list]: a list of label names
                rgba[ndarray]: shape=(n_label, 4)
                    The four elements in the second dimension are
                    red, green, blue, and alpha color components for label (between 0 and 1).
        """
        label_info = []
        for named_map in self.header.get_index_map(0).named_maps:
            label_dict = {'key': [], 'label': [], 'rgba': []}
            for k, v in named_map.label_table.items():
                label_dict['key'].append(k)
                label_dict['label'].append(v.label)
                label_dict['rgba'].append(v.rgba)
            label_dict['rgba'] = np.asarray(label_dict['rgba'])
            label_info.append(label_dict)

        return label_info

    @property
    def volume(self):
        return self.header.get_index_map(1).volume

    def brain_models(self, structures=None):
        """
        get brain model from cifti file
        Parameter:
        ---------
        structures: list of str
            Each structure corresponds to a brain model.
            If None, get all brain models.
        Return:
        ------
            brain_models: list of Cifti2BrainModel
        """
        brain_models = list(self.header.get_index_map(1).brain_models)
        if structures is not None:
            if not isinstance(structures, list):
                raise TypeError("The parameter 'structures' must be a list")
            b_strus = [bm.brain_structure for bm in brain_models]
            brain_models = [brain_models[b_strus.index(s)] for s in structures]
        return brain_models

    def map_names(self, map_indices=None):
        """
        get map names
        Parameters:
        ----------
        map_indices: sequence of integer
            Specify which map names should be got.
            If None, get all map names
        Return:
        ------
        map_names: list of str
        """
        named_maps = list(self.header.get_index_map(0).named_maps)
        if named_maps:
            if map_indices is None:
                map_names = [nm.map_name for nm in named_maps]
            else:
                map_names = [named_maps[i].map_name for i in map_indices]
        else:
            map_names = []
        return map_names

    def label_tables(self, map_indices=None):
        """
        get label tables
        Parameters:
        ----------
        map_indices: sequence of integer
            Specify which label tables should be got.
            If None, get all label tables.
        Return:
        ------
        label_tables: list of Cifti2LableTable
        """
        named_maps = list(self.header.get_index_map(0).named_maps)
        if named_maps:
            if map_indices is None:
                label_tables = [nm.label_table for nm in named_maps]
            else:
                label_tables = [named_maps[i].label_table for i in map_indices]
        else:
            label_tables = []
        return label_tables

    def get_stru_pos(self, structure):
        """
        For a certain brain structure:
        1. Get its position in the cifti data:
            offset, count
        2. Get its position in the intact map:
            map_shape, index2v
        Args:
            structure (str): brain structure
                Can be found by self.brain_structures
        Returns:
            offset (int): data offset
                Offset of the brain structure in the cifti data.
            count (int): the number of data points
                Data point count of the brain structure in the cifti data.
            map_shape (tuple): the shape of the map
                If brain model type is SURFACE, the shape is (n_vertex,).
                If brain model type is VOXELS, the shape is (i_max, j_max, k_max).
            index2v (list): index to vertex or voxel
                index2v[cifti_data_index] == vertex number or voxel position
        """
        bm = self.brain_models([structure])[0]
        offset, count = bm.index_offset, bm.index_count
        if bm.model_type == 'CIFTI_MODEL_TYPE_SURFACE':
            map_shape = (bm.surface_number_of_vertices,)
            index2v = list(bm.vertex_indices)
        elif bm.model_type == 'CIFTI_MODEL_TYPE_VOXELS':
            # This function have not been verified visually.
            map_shape = self.volume.volume_dimensions
            index2v = list(bm.voxel_indices_ijk)
        else:
            raise RuntimeError(f'Not supported brain model: {bm.model_type}')

        return offset, count, map_shape, index2v

    def get_data(self, structure=None, zeroize=False):
        """
        get data from cifti file
        Parameters
        ----------
        structure: str
            One structure corresponds to one brain model.
            specify which brain structure's data should be extracted
            If None, get all structures, meanwhile ignore parameter 'zeroize'.
        zeroize: bool
            If true, get data after filling zeros for the missing vertices/voxels.
        Return
        ------
        data: numpy array
            If zeroize is False, the data's shape is (n_map, n_index).
            If zeroize is True and brain model type is SURFACE,
                the data's shape is (n_map, n_vertex).
            If zeroize is True and brain model type is VOXELS,
                the data's shape is (n_map, i_max, j_max, k_max).
        """
        _data = self.full_data.get_fdata()
        if structure is not None:
            bm = self.brain_models([structure])[0]
            offset, count, map_shape, index2v = self.get_stru_pos(structure)
            _data = _data[:, offset:offset+count]

            if zeroize:
                data_shape = (_data.shape[0],) + map_shape
                data = np.zeros(data_shape, _data.dtype)
                if bm.model_type == 'CIFTI_MODEL_TYPE_SURFACE':
                    data[:, index2v] = _data
                elif bm.model_type == 'CIFTI_MODEL_TYPE_VOXELS':
                    index2v = np.array(index2v)
                    data[:, index2v[:, 0], index2v[:, 1], index2v[:, 2]] = _data
                else:
                    raise RuntimeError(f'Not supported brain model: {bm.model_type}')
            else:
                data = _data
        else:
            data = _data

        return data

In [4]:
cii = CiftiReader('template.dtseries.nii')
_, _, _, index2v = cii.get_stru_pos('CIFTI_STRUCTURE_CORTEX_LEFT')

In [6]:
save_path = '/nfs/z1/userhome/GongZhengXin/NVP/NaturalObject/data/code/nodanalysis/localizer-zscore-nii'
sub='sub-01'
spatialcode = 's4'
for roi in ['vwfa-1', 'vwfa-2','owfa','body', 'ffa-1', 'ffa-2', 'ofa', 'place']:
    nii =nib.load(pjoin(save_path,f'rg_{sub}_localizer_{roi}.nii')).get_fdata()
    nii = np.squeeze(nii)
    cii = nii[index2v]
    roi_vertices = np.where(cii==1)
    roi_mask = np.zeros(91282)
    roi_mask[roi_vertices] = 1
    save_ciftifile(roi_mask, pjoin(save_path, f'{sub}_{spatialcode}-{roi}.dtseries.nii'), 'template.dtseries.nii')

In [17]:
# draw ROI
sub = 'sub-01'
spatialcode = '_s4'
local_nii_path = '/nfs/z1/userhome/GongZhengXin/NVP/NaturalObject/data/code/nodanalysis/localizer-zscore-nii'
if spatialcode:
    nii_file = f'{sub}_localizer_zscore{spatialcode}.dtseries.nii'
else:
    nii_file = f'{sub}_localizer_zscore.dtseries.nii'
# surf_gii_path = '/nfs/z1/userhome/GongZhengXin/NVP/NaturalObject/data/bold/derivatives/ciftify/sub-01/MNINonLinear/fsaverage_LR32k/'
# gii_file = 'sub-01.L.very_inflated.32k_fs_LR.surf.gii'

# gii = nib.load(pjoin(surf_gii_path, gii_file))
# triangles = gii.agg_data('triangle')

nii = nib.load(pjoin(local_nii_path, nii_file))
data = nii.get_fdata()


In [8]:
(1-np.isnan(data)).sum()

0

In [18]:
threshold = 2.33
char_mask = data[0,:] > threshold
char_vertices = np.where(char_mask==1)[0]
body_vertices = np.where((data[1,:] > threshold) == 1)[0]
face_vertices = np.where((data[2,:] > threshold) == 1)[0]
place_vertices = np.where((data[3,:] > threshold) == 1)[0]

In [19]:
##>> handle the overlap boundary
overlap_on_face = set(face_vertices) & (set(char_vertices) | set(body_vertices) | set(place_vertices))
char_vertices = set(char_vertices) - overlap_on_face
body_vertices = set(body_vertices) - overlap_on_face
place_vertices = set(place_vertices) - overlap_on_face

overlap_on_body = set(body_vertices) & (set(char_vertices) | set(place_vertices))
char_vertices = char_vertices - overlap_on_body
place_vertices = place_vertices - overlap_on_body

overlap_on_place = set(place_vertices) & set(char_vertices)
char_vertices = char_vertices - overlap_on_place

In [20]:
roi_map = np.nan*np.zeros(91282)
roi_map[list(body_vertices)] = 8
roi_map[list(face_vertices)] = 4
roi_map[list(char_vertices)] = -1
roi_map[list(place_vertices)] = -5
save_path = '/nfs/z1/userhome/GongZhengXin/NVP/NaturalObject/data/code/nodanalysis/localizer-zscore-nii'
if spatialcode:
    save_ciftifile(roi_map, pjoin(save_path, f'{sub}_roi_thres2.33{spatialcode}.dtseries.nii'), 'template.dtseries.nii')
else:
    save_ciftifile(roi_map, pjoin(save_path, f'{sub}_roi_thres2.33.dtseries.nii'), 'template.dtseries.nii')

In [25]:
# roi_vertices = char_vertices
# mesh_indices = []
# for i in range(triangles.shape[0]):
#     cur_ver = triangles[i, :]
#     if np.all([_ in roi_vertices for _ in cur_ver]):
#         mesh_indices.append(i)

In [112]:
[425+_*258 for _ in [0,1,2,3]]

[425, 683, 941, 1199]

In [35]:
a = [1,4,5,6,8,9,10,11]
print(np.diff(a))
np.r_[1,np.diff(a)]

[3 1 1 2 1 1 1]


array([1, 3, 1, 1, 2, 1, 1, 1])

In [14]:
from nod_utils import save_ciftifile
ver1 = np.unique([ _ for _ in triangles if 7322 in _])
ver2 = np.unique([ _ for _ in triangles if 9031 in _])
bmap = np.nan*np.zeros(91282)
bmap[ver1] = 100
bmap[ver2] = 10
save_path = '/nfs/z1/userhome/GongZhengXin/NVP/NaturalObject/data/code/nodanalysis/inter-subject-animacymap'
save_ciftifile(bmap, pjoin(save_path, f'verticeIndex.dtseries.nii'), 'template.dtseries.nii')


In [11]:
ver1

array([7276, 7277, 7321, 7322, 7323, 7366, 7367], dtype=int32)

In [2]:
sub = ny.freesurfer_subject('fsaverage')
# /nfs/z1/userhome/GongZhengXin/NVP/NaturalObject/data/bold/derivatives/freesurfer/sub-08

In [3]:
file = h5py.File('/nfs/z1/userhome/GongZhengXin/NVP/NaturalObject/data/code/nodanalysis/prfresults.mat','r')
file.keys()
ciftifsaverageix = file['ciftifsaverageix'][:]
ciftifsaveragebad = file['ciftifsaveragebad'][:]
ix = np.squeeze(ciftifsaverageix.astype(np.int64)) - 1
bad_id = np.squeeze(ciftifsaveragebad.astype(np.int64))

In [83]:
import nibabel as nib
funcloc_path = '/nfs/z1/userhome/GongZhengXin/NVP/NaturalObject/data/code/nodanalysis/localizer-zscore-nii'
subj = 'sub-04'

pvalue = nib.load(pjoin(funcloc_path, f'{subj}_localizer_single-pvalue.dtseries.nii')).get_fdata()
logpvalue = nib.load(pjoin(funcloc_path, f'{subj}_localizer_single-logpvalue.dtseries.nii')).get_fdata()

pvalue = pvalue[:,ix]
pvalue[:, bad_id] = np.nan
logpvalue = logpvalue[:, ix]
logpvalue[:, bad_id] = np.nan

# get curvature
curv_path = f'/nfs/z1/userhome/GongZhengXin/NVP/NaturalObject/data/bold/derivatives/ciftify/{subj}/MNINonLinear/fsaverage_LR32k'
curv_file = f'{subj}.curvature.32k_fs_LR.dscalar.nii'
curvature = np.squeeze(nib.load(pjoin(curv_path, curv_file)).get_fdata())
fsa_curvature = curvature[ix]
fsa_curvature[bad_id] = np.nan

print(pvalue.shape)

(5, 327684)


In [106]:
# Plot only on the cortical surface (i.e., exclude the Corpus callosum).
# The 'cortex_label' property stores True values for cortex and False
# values for the medial wall.
import pythreejs
import ipywidgets as widgets
lh_valuemap = -logpvalue[:,0:163842]
fig = ipv.figure()
ny.cortex_plot(sub.lh, surface='inflated', color=lh_valuemap[0,:],figure=fig,
               cmap='YlOrRd', vmin=2, vmax=12, underlay=-fsa_curvature[0:163842], mask=(lh_valuemap[0,0:163842]>=2), 
               view='front') #

# ipv.pylab.style.box_on()
# ipv.pylab.style.axes_on()
# # ipv.pylab.quiver(0,0,0,1,0,0, color='red')
# # ipv.pylab.quiver(0,0,0,0,1,0, color='blue')
# # ipv.pylab.quiver(0,0,0,0,0,1, color='green')
# ipv.pylab.xyzlabel('x','y','z')
ipv.show()

In [105]:
rh_valuemap = -logpvalue[:,163842::]
fig_rh = ipv.figure()
ny.cortex_plot(sub.rh, surface='inflated', color=rh_valuemap[0,:],figure=fig_rh,
               cmap='YlOrRd', vmin=1, vmax=12, underlay=-fsa_curvature[163842::], mask=(rh_valuemap[0,163842::]>=1), 
               view='front') #

Figure(camera=PerspectiveCamera(fov=0.644570721372708, position=(0.0, 100.0, 0.0), projectionMatrix=(1.0, 0.0,…

In [8]:
ny.cortex_plot(sub.lh, surface='inflated', color='thickness',
               cmap='hot', vmin=2, vmax=6,
               mask=('thickness', 0, np.inf),
               underlay='curvature')


Figure(camera=PerspectiveCamera(fov=0.644570721372708, position=(0.0, -100.0, 0.0), projectionMatrix=(1.0, 0.0…

In [63]:
help(ipv.figure)

Help on function figure in module ipyvolume.pylab:

figure(key=None, width=400, height=500, lighting=True, controls=True, controls_vr=False, controls_light=False, debug=False, **kwargs)
    Create a new figure (if no key is given) or return the figure associated with key
    
    :param key: Python object that identifies this figure
    :param int width: pixel width of WebGL canvas
    :param int height:  .. height ..
    :param bool lighting: use lighting or not
    :param bool controls: show controls or not
    :param bool controls_vr: show controls for VR or not
    :param bool debug: show debug buttons or not
    :return: :any:`Figure`



In [12]:
###>> save to localizer-zscore-nii
from nod_utils import save_ciftifile
from scipy.stats import norm
save_path = '/nfs/z1/userhome/GongZhengXin/NVP/NaturalObject/data/code/nodanalysis/localizer-zscore-nii'
subject = 'sub-08'
spatial_smoooth_code = '_s4'
zscore = np.load(f'/nfs/z1/userhome/GongZhengXin/NVP/NaturalObject/data/bold/derivatives/beta/{subject}/{subject}_localizer-z_score{spatial_smoooth_code}.npy')
save_ciftifile(zscore, pjoin(save_path, f'{subject}_localizer_zscore{spatial_smoooth_code}.dtseries.nii'), 'template.dtseries.nii')

# log_zscore = norm.logsf(zscore)/np.log(10)
# log_zscore[log_zscore>(np.log(0.1)/np.log(10))] = np.nan
# save_ciftifile(log_zscore, pjoin(save_path, f'{subject}_localizer_single-logpvalue.dtseries.nii'), 'template.dtseries.nii')
# p_zscore = norm.sf(zscore)
# p_zscore[p_zscore>0.1] = np.nan
# save_ciftifile(p_zscore, pjoin(save_path, f'{subject}_localizer_single-pvalue.dtseries.nii'), 'template.dtseries.nii')

(5, 91282)

In [22]:
log_zscore[log_zscore>(np.log(0.1)/np.log(10))] = np.nan

-10.461752333089393

In [13]:
np.log(np.min(norm.sf(zscore[1])))

-24.089074968767317